In [2]:
from detr.explainer import DetrExplainer
from pathlib import Path
from PIL import Image

import torch
import transformers as tr

In [3]:
def load_detr_from_ckpt(path_ckpt: Path):
    ckpt = torch.load(path_ckpt)
    model = tr.DetrForObjectDetection.from_pretrained(
            "facebook/detr-resnet-50",
            num_labels=ckpt['hyper_parameters']['max_class_id'] + 1,
            ignore_mismatched_sizes=True)
    
    state_dict = ckpt['state_dict']
    state_dict = {k.replace('model.', '', 1): v for k, v in state_dict.items()} 
    state_dict = {k.replace('detr.', 'model.', 1): v for k, v in state_dict.items()}
    state_dict = {k.replace('model.class_labels_classifier.', 'class_labels_classifier.', 1): v for k, v in state_dict.items()}
    state_dict = {k.replace('model.bbox_predictor.', 'bbox_predictor.', 1): v for k, v in state_dict.items()}
    model.load_state_dict(state_dict)
    
    return model

In [4]:
PATH_CKPT = Path(r'./resources/checkpoints/model.ckpt')

model = load_detr_from_ckpt(PATH_CKPT)
processor = tr.DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")

Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DetrForObjectDetection were not initialized from the model checkpoin

In [5]:
detr_explainer = DetrExplainer(model, processor, device='cuda')

categories=[
    'road',
    'sidewalk',
    'building',
    'wall',
    'fence',
    'pole',
    'traffic light',
    'traffic sign',
    'vegetation',
    'terrain',
    'sky',
    'person',
    'rider',
    'car',
    'truck',
    'bus',
    'train',
    'motorcycle',
    'bicycle'
]

no_object_id = 20

In [7]:
img_paths = Path(r'./resources/images').glob('*.png')

for img_path in list(img_paths)[:1]:
    detr_explainer.explain(
        image=Image.open(img_path),
        no_object_id=no_object_id,
        categories=categories,
        output_dir=Path(r'./explanations') / img_path.stem,
        output_dir=None,
        threshold=0.9)

The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.
